In [1]:
import numpy as np

In [2]:
import random

In [3]:
import time

In [4]:
import threading

<h3>Generating random n random cities and distances between them.</h3>

<p><small>*The below mentioned code creates a random array of elements of size n^2.*<br>
    *It then makes the diagonal elements 0(considering the distance between a city and itself is 0)*<br>
    *The last section of code makes symmetric elements of array (Since, dist b/w n to m is equal to dist b/w m to n.)*</small></p>

In [5]:
n = 10
dist = np.array([random.sample(range(n**2),n**2)])
dist = dist.reshape(n,n)

for i in range(n):
    dist[i][i] = 0
    
for i in range(n):
    for j in range(n):
        if j < i:
            dist[i][j] = dist[j][i]

In [6]:
dist = np.array([[ 0, 82, 19, 39, 99, 73, 40, 34, 69, 81],
       [82,  0, 41, 63, 53, 60, 93, 27,  2, 10],
       [19, 41,  0, 26, 36, 67, 52, 87, 56, 94],
       [39, 63, 26,  0, 76, 57, 55, 35, 45, 11],
       [99, 53, 36, 76,  0, 64, 58, 28, 44, 29],
       [73, 60, 67, 57, 64,  0, 62,  3,  6, 75],
       [40, 93, 52, 55, 58, 62,  0, 16, 80, 47],
       [34, 27, 87, 35, 28,  3, 16,  0,  5, 91],
       [69,  2, 56, 45, 44,  6, 80,  5,  0,  1],
       [81, 10, 94, 11, 29, 75, 47, 91,  1,  0]])

In [7]:
dist

array([[ 0, 82, 19, 39, 99, 73, 40, 34, 69, 81],
       [82,  0, 41, 63, 53, 60, 93, 27,  2, 10],
       [19, 41,  0, 26, 36, 67, 52, 87, 56, 94],
       [39, 63, 26,  0, 76, 57, 55, 35, 45, 11],
       [99, 53, 36, 76,  0, 64, 58, 28, 44, 29],
       [73, 60, 67, 57, 64,  0, 62,  3,  6, 75],
       [40, 93, 52, 55, 58, 62,  0, 16, 80, 47],
       [34, 27, 87, 35, 28,  3, 16,  0,  5, 91],
       [69,  2, 56, 45, 44,  6, 80,  5,  0,  1],
       [81, 10, 94, 11, 29, 75, 47, 91,  1,  0]])

<h3>Declaring cities as 0-n</h3>

In [8]:
cities = [i for i in range(n)]

<h3>Declaring the size of population </h3>

In [9]:
POPULATION_SIZE = 1000

<h3>Defining the class Individual</h3>

<p><small>
    __Data this class contains :__ <br><br>
    *__chromosome__ :- it is list which contains the path*<br>
    *__fitness__ :- it is an integer which contains the fitness of an Individual object*<br><br><br>
    __Methods of this class : __<br><br>
    *__create_gnome ::__ <br>__Input__ : the class itself<br>__Return__ : a random path<br>  *
    *__mate ::__<br>__Input__ : two individual objects(One acts as an invoker)<br>__Return__:an object of Individual class which acts as child<br>*
    *__mutation ::__<br>__Input__: the object itself<br>__Return__: if probability allows, swaps two random cities in the path<br>*
    *__cal_fitness ::__<br>__Input__: the object itself<br>__Return__: it return the path length as fitness of Individual object<br>*
    
    
    
    
    
</small></p>

In [11]:
class Individual():
    
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.path_length = self.cal_path_length()
        self.fitness = self.cal_fitness()
        
    @classmethod
    def create_gnome(self):
        gnome = []
        temp = [i for i in cities]
        for i in cities:
            x = random.choice(temp)
            gnome.append(x)
            temp.remove(x)    
        return(gnome)
    
    def mate(self, par2):
        x_part = 3
        y_part = 6
        t1 = self.chromosome[x_part:y_part]
        t2 = par2.chromosome[x_part:y_part]
        child1 = []
                
        for i, j in zip(self.chromosome[0:x_part], par2.chromosome[0:x_part]):
            if i not in t2 :
                child1.append(i) 
            else:
                temp1 = random.choice(t1)
                while (temp1 in t2) or (temp1 in child1) :
                    temp1 = random.choice(t1)
                child1.append(temp1)
        
        child1 = child1+t2
               
        length = len(cities)
        for i, j in zip(self.chromosome[y_part:length], par2.chromosome[y_part:length]):
            if (i not in t2):
                child1.append(i)
            else:
                temp1 = random.choice(t1)
                while temp1 in t2 or temp1 in child1:
                    temp1 = random.choice(t1)
                child1.append(temp1)
        return(Individual(child1))
    
    def mutation(self):
        global cities
        prob = random.random()
        mutation_rate = 0.4
        if prob < mutation_rate:
            x = random.choice(cities)
            y = random.choice(cities)
            temp = self.chromosome[x]
            self.chromosome[x] = self.chromosome[y]
            self.chromosome[y] = temp
            

    def cal_fitness(self):
        return (self.path_length)**2
    
    
    def cal_path_length(self):
        path = 0
        length = len(self.chromosome)
        for i in range(length-1):
            path = path + dist[self.chromosome[i], self.chromosome[i+1]]
        
        path = path + dist[self.chromosome[length-1],self.chromosome[0]]
        
        return path

    @classmethod   
    def selection(self, population):
        while True:
            max_fitness = 1000000000000
            par = random.choice(population)
            prob = random.randint(0, max_fitness)
        
            if(prob > par.fitness):
                return par

<h3>Initializing some variables </h3>

In [12]:
population = []
new_generation = []

<h3>Function which each thread calls </h3>

<p>Each thread adds a new child to the new_generation which its equal to the population_size</p>

In [13]:
def fun1():
    global population
    global POPULATION_SIZE
    global new_generation
            
    s = int((90*POPULATION_SIZE)/100)
    for _ in range(s):
        par1, par2 = Individual.selection(population),Individual.selection(population)
        child = par1.mate(par2)
        child.mutation()
        new_generation.append(child)
    population = new_generation

<h3>Defining the main function </h3>

<p>__In main, we follow the following steps : __<br>
   *->A random population of Individual objects is created.*<br>
   *->Objects are sorted according to their fitness values.*<br>
   *->The first 20% of sorted objects is directly sent to the new generation.*<br>
   *->The remaining childs are created by randomly choosing parents from the population and added to next generation.<br>*
    </p>

In [14]:
def main():
    global POPULATION_SIZE
    global cities
    global population
    generation = 1
    found = False
    
    population = [Individual(Individual.create_gnome()) for _ in range(POPULATION_SIZE)]
        
    while not found:
        population = sorted(population, key = lambda x : x.fitness)
     
        if population[0].path_length <= 200 or generation == 100:
            found = True
            break
        
        s = int((10*POPULATION_SIZE)/100)
        new_generation.extend(population[:s])
        
        t1 = threading.Thread(target = fun1)
        t2 = threading.Thread(target = fun1)
        t3 = threading.Thread(target = fun1)
        t4 = threading.Thread(target = fun1)
        t1.start()
        t2.start()
        t3.start()
        t4.start()
        t1.join()
        t2.join()
        t3.join()
        t4.join()
        
        print("Generation: {}\tPath: {}\tPath_Length: {}".format(generation,population[0].chromosome,population[0].path_length))
        generation += 1
    
    print("Generation: {}\tPath: {}\tPath_Length: {}".format(generation,population[0].chromosome,population[0].path_length))

    generation += 1
       



<h3>Invoking the main function</h3>

In [15]:
avg = []

In [24]:
if __name__ == "__main__":
    s = time.clock()
    main()
    avg.append(time.clock()-s)
    print(time.clock()-s)
    


Generation: 1	Path: [5, 0, 2, 3, 9, 1, 8, 4, 6, 7]	Path_Length: 262
Generation: 2	Path: [4, 6, 7, 5, 8, 1, 9, 3, 0, 2]	Path_Length: 200
0.2003962816648137


In [25]:
sum(avg)/len(avg)

0.19726482515264704

<h3>Issues:</h3>
<p>*->Gets stuck at a local minimum.<br>*
   </p>

<p>Parameters to be tuned : <br>
    ->population_size<br>
    ->mutation_rate<br>
    ->percentage population to be passed to next generation<br>
    ->stopping criteria(minimum path_length or max_generations)<br>

In [128]:
len(population)

300